In [14]:
import fitz

In [34]:
def search_text_and_get_coordinates(pdf_path, search_text):
    document = fitz.open(pdf_path)
    
    start_text_coordinates = (float('inf'), float('inf'))
    end_text_coordinates = (float('-inf'), float('-inf'))
    
    for page_number in range(len(document)):
        page = document.load_page(page_number)
        
        text_instances = page.search_for(search_text)
        
        for inst in text_instances:
            x0, y0, x1, y1 = inst
            if start_text_coordinates is None:
                start_text_coordinates = (x0, y0)
            else:
              # choose the smallest x0 and y0
              start_text_coordinates = (min(start_text_coordinates[0], x0), min(start_text_coordinates[1], y0))
        
            if end_text_coordinates is None:
                end_text_coordinates = (x1, y1)
            else:
              # choose the largest x1 and y1
              end_text_coordinates = (max(end_text_coordinates[0], x1), max(end_text_coordinates[1], y1))
              
    return start_text_coordinates, end_text_coordinates

pdf_path = 'Structured_note_termsheet/cct-linked-structured-note.pdf'

search_text = 'CCT LINKED STRUCTURED NOTE  FINAL TERM-SHEET1 Principal Terms and Conditions'

search_text_and_get_coordinates(pdf_path, search_text)

((209.08999633789062, 103.81997680664062),
 (411.1659851074219, 154.0640106201172))

In [ ]:
{
  "text": "Status of the Notes: The Notes are direct, unsecured and unsubordinated obligations of the Issuer and rank pari passu without any preference among themselves and (save for certain obligations required to be preferred by law) equally with all other unsecured obligations (other than subordinated obligations, if any) of the Issuer, from time to time outstanding.",
  "start_text_coordinates": (float('inf'), float('inf')),
  "end_text_coordinates": (float('-inf'), float('-inf'))
},
{
  "expected_output": "Status of the Notes: The Notes are direct, unsecured and unsubordinated obligations of the Issuer and rank pari passu without any preference among themselves and (save for certain obligations required to be preferred by law) equally with all other unsecured obligations (other than subordinated obligations, if any) of the Issuer, from time to time outstanding.",
}